In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F


import nltk
nltk.download('punkt')         # only needed first time
nltk.download('stopwords')     # only needed first time
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report
from copy import deepcopy
import joblib
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
# hyperparameters
#batch_size = 900 # how many independent sequences will we process in parallel?
block_size = 149 # what is the maximum context length for predictions? This is decided based on value in max_length
max_iters = 20
eval_interval = 10
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 20
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
num_classes = 2
PATH_MODEL = '/content/drive/My Drive/Models/model_transformer_sentiment_batched.pth'
best_val_loss = float('inf')
# ------------

In [4]:
#Need some more work to implement multiple files
"""
# Read all the *labelled.txt files in the curent directory
dfs = []

# Specify the file path the Google Drive
directory_path = '/content/drive/My Drive/Data/'

for filename in os.listdir(directory_path):
    if filename.endswith('labelled.txt'):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)
        # Read CSV file into a DataFrame
        df = pd.read_csv(file_path, delimiter='\t', header=None)
        # Append DataFrame to list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
data = pd.concat(dfs, ignore_index=True)
"""

In [25]:
# Specify the file path the Google Drive
file_path = '/content/drive/My Drive/Data/amazon_cells_labelled.txt'

# Read the file using pandas
data = pd.read_csv(file_path, delimiter='\t', header=None)


In [5]:
def preprocess_pandas(data, columns):
    df_ = pd.DataFrame(columns=columns)
    data['Sentence'] = data['Sentence'].str.lower()
    data['Sentence'] = data['Sentence'].replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', regex=True)                      # remove emails
    data['Sentence'] = data['Sentence'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', regex=True)    # remove IP address
    data['Sentence'] = data['Sentence'].str.replace('[^\w\s]','')                                                       # remove special characters
    data['Sentence'] = data['Sentence'].replace('\d', '', regex=True)                                                   # remove numbers
    for index, row in data.iterrows():
        word_tokens = word_tokenize(row['Sentence'])
        filtered_sent = [w for w in word_tokens if not w in stopwords.words('english')]
        df_ = df_._append({
            "index": row['index'],
            "Class": row['Class'],
            "Sentence": " ".join(filtered_sent[0:])
        }, ignore_index=True)
    return data

In [26]:
data.columns = ['Sentence', 'Class']
data['index'] = data.index                                          # add new column index
columns = ['index', 'Class', 'Sentence']
data = preprocess_pandas(data, columns)                             # pre-process

training_data, validation_data, training_labels, validation_labels = train_test_split( # split the data into training, validation, and test splits
    data['Sentence'].values.astype('U'),
    data['Class'].values.astype('int32'),
    test_size=0.10,
    random_state=0,
    shuffle=True
)

test_data, validation_data, test_labels, validation_labels = train_test_split( # split the data into training, validation, and test splits
    validation_data,
    validation_labels,
    test_size=0.50,
    random_state=0,
    shuffle=True
)

In [27]:
# here are all the unique characters that occur in this text
concatenated_string = data['Sentence'].str.cat()
chars = sorted(list(set(concatenated_string)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [8]:
def encode_text(text):
    if isinstance(text, str):  # Check if input is a single string
        return [stoi[c] for c in text]
    elif isinstance(text, np.ndarray):  # Check if input is a NumPy array
        return [encode_text(s) for s in text]
    elif isinstance(text, list):  # Check if input is a list of strings
        return [encode_text(s) for s in text]
    else:
        raise TypeError("Input must be a string or a list of strings")

In [28]:
print(''.join(chars))
vocab_size

 !"#$%&'()*+,-./:;?[]abcdefghijklmnopqrstuvwxyz


47

In [10]:
# Define a custom dataset
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [29]:
#encode training data
encoded_training_data = encode_text([training_data[i] for i in range(len(training_data))])

# Determine the maximum length
max_length = max(len(seq) for seq in encoded_training_data)

# Pad or truncate per maximum length
padded_training_data = [seq + [0] * (max_length - len(seq)) for seq in encoded_training_data]

# Convert to a PyTorch tensor
encoded_training_data_tensor = torch.tensor(padded_training_data, dtype=torch.long)

# Convert labels to tensor
training_labels = torch.tensor(training_labels, dtype=torch.long)

In [30]:
max_length   # this decides block_size

149

In [31]:
#Encode validation data
encoded_validation_data = encode_text([validation_data[i] for i in range(len(validation_data))])

# Pad or truncate per maximum length
padded_validation_data = [seq + [0] * (max_length - len(seq)) for seq in encoded_validation_data]

# Convert to a PyTorch tensor
encoded_validation_data_tensor = torch.tensor(padded_validation_data, dtype=torch.long)


# Convert labels to tensor
validation_labels = torch.tensor(validation_labels, dtype=torch.long)


In [32]:
#Encode test data
encoded_test_data = encode_text([test_data[i] for i in range(len(test_data))])

# Pad or truncate per maximum length
padded_test_data = [seq + [0] * (max_length - len(seq)) for seq in encoded_test_data]

# Convert to a PyTorch tensor
encoded_test_data_tensor = torch.tensor(padded_test_data, dtype=torch.long)


# Convert labels to tensor
test_labels = torch.tensor(test_labels, dtype=torch.long)

In [33]:
# Create a dataset instance
train_dataset = MyDataset(encoded_training_data_tensor, training_labels)
val_dataset = MyDataset(encoded_validation_data_tensor, validation_labels)
test_dataset = MyDataset(encoded_test_data_tensor, test_labels)

In [34]:
# Hyperparams. Set these to reasonable values
BATCH_SIZE = 50
SHUFFLE = 2


# Load data in the loader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

## Transformer adapted to do sentiment analysis

In [44]:
@torch.no_grad()
def estimate_loss(best_val_loss):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):

            if split == 'train':
              for inputs, label in train_loader:
                X, Y = inputs, label

                #X, Y = get_batch(split)
                logits, _ = model(X)
                loss = criterion(logits, Y)
                losses[k] = loss.item()

            else:
              for inputs, label in val_loader:
                X, Y = inputs, label

                #X, Y = get_batch(split)
                logits, _ = model(X)
                loss = criterion(logits, Y)
                losses[k] = loss.item()

                # when best model , save it
                if loss < best_val_loss:
                    # save the best loss
                    best_val_loss = loss
                    # Save the model
                    torch.save(model.state_dict(), PATH_MODEL)

        out[split] = losses.mean()
    model.train()
    return out, best_val_loss

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        #wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)    # commented this masking for sentiment analysis so that every token can communicate with every other token
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        #self.lm_head = nn.Linear(n_embd, vocab_size)
        self.sentiment_classifier = nn.Linear(n_embd, num_classes) # classfier layer

    def forward(self, idx, targets=None):
        #print(idx.shape)
        B, T = idx.shape


        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        #logits = self.lm_head(x) # (B,T,vocab_size)  # this is from nanogpt implementation (decoder - generates text)
        logits = self.sentiment_classifier(x[:, -1, :])  # Use only the last output of the model

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    # Generate is not used for sentiment analysis.
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [45]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

best_val_loss_prev = float('inf')
best_val_loss_current = float('inf')

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses, best_val_loss_current = estimate_loss(best_val_loss_prev)
        best_val_loss_prev = best_val_loss_current
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    for inputs, label in train_loader:
      xb, yb = inputs, label

      # evaluate the loss
      logits, _ = model(xb)
      loss = criterion(logits, yb)
      optimizer.zero_grad(set_to_none=True)
      loss.backward()
      optimizer.step()

# Save the model
#PATH = '/content/drive/My Drive/Models/model_transformer_sentiment.pth'
#torch.save(model.state_dict(), PATH)

0.21197 M parameters
step 0: train loss 0.6990, val loss 0.7057
step 10: train loss 0.5646, val loss 0.7290
step 19: train loss 0.2335, val loss 1.2438


### Test the model on test set
With 200 iteration - Result is 78% accuracy  
With 20 iteration - Result is 46% accuracy

In [46]:
# test the model
correct = 0
total = 0

# Hyperparams. Set these to reasonable values
BATCH_SIZE = 50
SHUFFLE = 2
LEARNING_RATE = 0.001

# Load data in the loader
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

# load the model
model = BigramLanguageModel()
m = model.to(device)
#PATH = '/content/drive/My Drive/Models/model_transformer_sentiment.pth'
model.load_state_dict(torch.load(PATH_MODEL))

with torch.no_grad():
    for inputs, label in test_loader:
      xb, yb = inputs, label

      # evaluate the loss
      logits, _ = model(xb)
      _, predicted = torch.max(logits.data,1)
      total += yb.size(0)
      correct += (predicted==yb).sum().item()

accuracy = correct/total * 100
print(f'Accuracy {accuracy} %')

Accuracy 46.0 %


### Bot that accepts user input, analyses it and responds to the user based on it analysis. Bot uses previously trained model for its analysis.

In [47]:
# Define a function to get a response for a given prompt
def get_response(inp,model):

    with torch.no_grad():
        output, _ = model(inp)
        _, predicted = torch.max(output.data,1)
    #print(output)
    #print(predicted)

    if predicted == torch.tensor([0]):
        return 'Negative'
    else:
        return 'Positive'


# Function to pre-process input data
def preprocess_user_input(prompt):
    # get data, pre-process
    data = pd.DataFrame([[prompt, None]], columns=['Sentence', 'Class'])
    data['index'] = data.index                                          # add new column index
    columns = ['index', 'Class', 'Sentence']
    data = preprocess_pandas(data, columns)
    data['Sentence'].values.astype('U')



    #encode user input
    encoded_user_input = encode_text([data['Sentence'][i] for i in range(len(data['Sentence']))])

    # Pad or truncate per maximum length
    padded_user_input = [seq + [0] * (max_length - len(seq)) for seq in encoded_user_input]

    # Convert to a PyTorch tensor
    encoded_user_input_tensor = torch.tensor(padded_user_input, dtype=torch.long)

    return encoded_user_input_tensor


# Main function to run the chatbot
def main():
    print("Bot: Hello! How are you feeling today? I will tell you if it is positive or negative.")

    while True:
        user_input = input("User: ")

        if user_input.lower() == 'exit':
            print("Bot: Goodbye!")
            break

        # Preprocess the input
        inp_x_tensor = preprocess_user_input(user_input)
        #print(inp_x_tensor)

        # Load the model
        model = BigramLanguageModel()
        m = model.to(device)
        PATH = '/content/drive/My Drive/Models/model_transformer_sentiment.pth'
        model.load_state_dict(torch.load(PATH))

        # Use the trained model to get the bot response. Bot responds with its analysis indicating wheter user input is positive or negative
        response = get_response(inp_x_tensor, model)
        print("Bot:", response)

if __name__ == "__main__":
    main()

Bot: Hello! How are you feeling today? I will tell you if it is positive or negative.
User: good
Bot: Positive
User: bad
Bot: Negative
User: thanks
Bot: Negative
User: negative
Bot: Positive
User: oh god
Bot: Positive
User: mobile phone
Bot: Negative
User: no
Bot: Negative
User: why not
Bot: Negative
User: thanks
Bot: Negative
User: thank you
Bot: Negative
User: great
Bot: Positive
User: short
Bot: Negative
User: exit
Bot: Goodbye!
